In [1]:
#Building a federated learning model

#Importing the libraries
import torch as th
from torch import nn, optim
import torch.nn.functional as F
import syft as sy
hook = sy.TorchHook(th)

W0710 16:16:02.953719 140577556596544 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.1-dev20190517). Fix this by compiling custom ops.
W0710 16:16:02.968107 140577556596544 deprecation_wrapper.py:119] From /home/ayush/anaconda3/lib/python3.7/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
#Define the Dataset
data = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [3]:
#Define the workers
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [4]:
# Sending data and target to workers
bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)
alice_data = data[2:4].send(alice)
alice_target = target[2:4].send(alice)

In [5]:
#Creating a model
model = nn.Linear(2, 1)

In [6]:
epochs = 20

for e in range(epochs):
    
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)
    
    bob_optim = optim.SGD(bob_model.parameters(), lr = 0.01)
    alice_optim = optim.SGD(alice_model.parameters(), lr = 0.01)
    
    for i in range(5):
        #Train bob model
        bob_optim.zero_grad()

        bob_preds = bob_model(bob_data)
        bob_loss = ((bob_preds - bob_target)**2).sum()
        bob_loss.backward()
        bob_optim.step()

        bob_loss = bob_loss.get().data

        #Train alice model
        alice_optim.zero_grad()

        alice_preds = alice_model(alice_data)
        alice_loss = ((alice_preds - alice_target)**2).sum()
        alice_loss.backward()
        alice_optim.step()

        alice_loss = alice_loss.get().data
    
    print("Bob loss : ", bob_loss, "Alice loss : ", alice_loss)
    
    alice_model.move(bob)
    
    with th.no_grad():
        model.weight.set_(((alice_model.weight.data + bob_model.weight.data)/2).get())
        model.bias.set_(((alice_model.bias.data + bob_model.bias.data)/2).get())

Bob loss :  tensor(0.0593) Alice loss :  tensor(0.4032)
Bob loss :  tensor(0.0184) Alice loss :  tensor(0.2837)
Bob loss :  tensor(0.0036) Alice loss :  tensor(0.2084)
Bob loss :  tensor(0.0003) Alice loss :  tensor(0.1595)
Bob loss :  tensor(0.0017) Alice loss :  tensor(0.1268)
Bob loss :  tensor(0.0043) Alice loss :  tensor(0.1041)
Bob loss :  tensor(0.0070) Alice loss :  tensor(0.0880)
Bob loss :  tensor(0.0091) Alice loss :  tensor(0.0761)
Bob loss :  tensor(0.0106) Alice loss :  tensor(0.0671)
Bob loss :  tensor(0.0116) Alice loss :  tensor(0.0600)
Bob loss :  tensor(0.0121) Alice loss :  tensor(0.0544)
Bob loss :  tensor(0.0122) Alice loss :  tensor(0.0496)
Bob loss :  tensor(0.0121) Alice loss :  tensor(0.0457)
Bob loss :  tensor(0.0118) Alice loss :  tensor(0.0422)
Bob loss :  tensor(0.0114) Alice loss :  tensor(0.0392)
Bob loss :  tensor(0.0110) Alice loss :  tensor(0.0365)
Bob loss :  tensor(0.0105) Alice loss :  tensor(0.0341)
Bob loss :  tensor(0.0100) Alice loss :  tensor(

In [7]:
preds = model(data)
loss =  ((preds - target)**2).sum()

In [8]:
print(preds)
print(target)
print(loss)

tensor([[0.8802],
        [0.9473],
        [0.0979],
        [0.1650]], grad_fn=<AddmmBackward>)
tensor([[1.],
        [1.],
        [0.],
        [0.]], requires_grad=True)
tensor(0.0540, grad_fn=<SumBackward0>)
